In [26]:
### import necessary libraries
import os
import ee
import sys
import json
import geemap
import sklearn
import numpy as np
import pandas as pd
import geopandas as gpd
from pprint import pprint
import matplotlib.pyplot as plt

# Sube un nivel en la estructura de directorios para llegar a la raíz del proyecto
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Ahora puedes importar la función refined_lee
from utils.refined_lee import refined_lee

ee.Authenticate()
ee.Initialize(project="ee-franciscofurey")

In [27]:
# Import the MODIS/061/MOD13Q1 image collection
modis = ee.ImageCollection("MODIS/061/MOD13Q1")

# Define the time of interest
start_year = 2010
end_year = 2024
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year, 12, 31)

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 8000,
  'palette': ['white', 'green']
}

# Set visualization parameters.
vis_params_2 = {
    'min': 0,  # Ajustado basado en el rango de salida
    'max': 0.8,  # Ajustado basado en el rango de salida
    'palette': ['white', 'red']
}

filtered = modis\
    .filterDate(start_date, end_date)\

# Get the 1 image from the collection
image = filtered.first()

map = geemap.Map()

def bitwise_extract(input, from_bit, to_bit):
    mask_size = ee.Number(1).add(to_bit).subtract(from_bit)
    mask = ee.Number(1).leftShift(mask_size).subtract(1)
    return input.rightShift(from_bit).bitwiseAnd(mask)

def mask_snow_and_clouds(image):
    summary_qa = image.select('SummaryQA')
    # Select pixels which are less than or equals to 1 (0 or 1)
    qa_mask = bitwise_extract(summary_qa, 0, 1).lte(1)
    masked_image = image.updateMask(qa_mask)
    return masked_image.copyProperties(image, ['system:index', 'system:time_start'])

# Mask the image and select the 'NDVI' band before visualizing
image_masked = ee.Image(mask_snow_and_clouds(image)).select('NDVI')

masked_col = filtered.map(mask_snow_and_clouds)
ndvi_col = masked_col.select('NDVI')

# Función para escalar los valores NDVI y copiar las propiedades deseadas
def scale_ndvi(image):
    scaled = image.divide(10000)
    return scaled.copyProperties(source=image, properties=['system:index', 'system:time_start'])

# Aplicar la función a cada imagen en la colección
ndvi_scaled = ndvi_col.map(scale_ndvi)

# Extract 1 image from the ndv_scaled collection
# Visualize it and add it into the map

# Add NDVI to map
map.addLayer(image.select('NDVI'), vis_params, 'MODIS NDVI')

# map.addLayer(ndvi_scaled, vis_params_2, 'MODIS NDVI (Masked & Scaled)')

# Filter the collection by date
filtered = modis.filter(ee.Filter.date(start_date, end_date))

def bitwise_extract(input, from_bit, to_bit):
    """
    Masks bits of an image between specified bit positions.
    """
    mask_size = ee.Number(1).add(to_bit).subtract(from_bit)
    mask = ee.Number(1).leftShift(mask_size).subtract(1)
    return input.rightShift(from_bit).bitwiseAnd(mask)

def mask_snow_and_clouds(image):
    """
    Masks snow and clouds based on the SummaryQA band.
    """
    summary_qa = image.select('SummaryQA')
    qa_mask = bitwise_extract(summary_qa, 0, 1).lte(1)
    masked_image = image.updateMask(qa_mask)
    return masked_image.copyProperties(image, ['system:index', 'system:time_start'])

# Apply cloud and snow masking
masked_col = filtered.map(mask_snow_and_clouds)

ndvi_col = masked_col.select('NDVI')

def scale_ndvi(image):
    """
    Scales NDVI values from MODIS.
    """
    scaled = image.divide(10000)
    return scaled.copyProperties(image, ['system:index', 'system:time_start'])

# Scale NDVI values
ndvi_scaled = ndvi_col.map(scale_ndvi)

In [29]:
years = ee.List.sequence(start_year, end_year)
months = ee.List.sequence(1, 12)

def calculate_monthly_images(years, months, ndvi_scaled):
    monthly_images = []
    for y in years.getInfo():
        for m in months.getInfo():
            image = ndvi_scaled \
                .filter(ee.Filter.calendarRange(y, y, 'year')) \
                .filter(ee.Filter.calendarRange(m, m, 'month')).mean() \
                .set({
                    'year': y,
                    'month': m,
                    'system:time_start': ee.Date.fromYMD(y, m, 1).millis()
                })
            monthly_images.append(image)
    return monthly_images

monthly_images = calculate_monthly_images(years, months, ndvi_scaled)


# Create a collection

monthly_images_col = ee.ImageCollection.fromImages(monthly_images)

# Filter the collection for the image for May 2015 and display the image on the map
image_2015_05 = monthly_images_col.filter(ee.Filter.eq('year', 2015)).filter(ee.Filter.eq('month', 5)).first()

print(monthly_images_col.size().getInfo()) 

# Filter the collection for the image for May 2015 and display the image on the map
image_2015_05 = monthly_images_col.filter(ee.Filter.eq('year', 2015)).filter(ee.Filter.eq('month', 5)).first()

# Add the image to the map
map.addLayer(image_2015_05, vis_params_2, 'May 2015 NDVI')

In [31]:
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…